# FactorCL on Multibench Datasets

##Preparation

In [1]:
!git clone https://github.com/pliang279/MultiBench

Cloning into 'MultiBench'...
remote: Enumerating objects: 6925, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 6925 (delta 62), reused 123 (delta 60), pack-reused 6789
Receiving objects: 100% (6925/6925), 51.06 MiB | 17.93 MiB/s, done.
Resolving deltas: 100% (4248/4248), done.


In [2]:
%cd MultiBench/

/content/MultiBench


In [3]:
!pip install gdown
#!gdown https://drive.google.com/uc?id=1_XdzdW8UNG1TTS6QcX10uhoS6N11OBit&export=download #MOSI data
#!gdown https://drive.google.com/u/0/uc?id=1KvKynJJca5tDtI5Mmp6CoRh9pQywH8Xp&export=download #AVMNIST
#!gdown https://drive.google.com/uc?id=180l4pN6XAv8-OAYQ6OrMheFUMwtqUWbz&export=download #MOSEI data
!gdown https://drive.google.com/uc?id=1EMBUmUL5B0PTncGx3L-sBElGOmjFBR_h&export=download #Sarcasm data
#!gdown https://drive.google.com/uc?id=1L5slPmYyhEVtwGyM1kgcFMjeBpXLZGT0&export=download #Humor data

#!sh /content/MultiBench/datasets/enrico/download_data.sh #Enrico

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Downloading...
From: https://drive.google.com/uc?id=1EMBUmUL5B0PTncGx3L-sBElGOmjFBR_h
To: /content/MultiBench/sarcasm.pkl
100% 208M/208M [00:01<00:00, 189MB/s]


In [4]:
!git clone https://github.com/pliang279/FactorCL

Cloning into 'FactorCL'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 37 (delta 14), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (37/37), 20.62 KiB | 1.87 MiB/s, done.


In [6]:
import os
import sys
from torch.utils.data import DataLoader
from sklearn.linear_model import LogisticRegression

from unimodals.common_models import Transformer, MLP

In [7]:
from datasets.affect.get_data import get_dataloader

# Create the training, validation, and test-set dataloaders. 
train_loader, valid_loader, test_loader = get_dataloader('/content/MultiBench/sarcasm.pkl', batch_size=128, data_type='sarcasm')

In [8]:
sys.path.append(os.getcwd())
#sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

In [9]:
%cd FactorCL

/content/MultiBench/FactorCL


In [10]:
from multibench_model import*

##FactorCL-SUP

In [ ]:
encoders = [Transformer(371, 40).cuda(), Transformer(300, 40).cuda()]
rus_model = RUSModel(encoders=encoders, feat_dims=[40, 40], y_ohe_dim=3).cuda()
train_rusmodel_sarcasm(rus_model, train_loader, num_epoch=100, num_club_iter=1)

In [15]:

train_embeds_x1 = np.concatenate([rus_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in train_loader])
train_embeds_x2 = np.concatenate([rus_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in train_loader])
train_embeds = np.concatenate([train_embeds_x1, train_embeds_x2], axis=1)
train_labels = np.concatenate([data[3].detach().cpu().numpy() for data in train_loader])
train_labels = sarcasm_label(train_labels)
  
test_embeds_x1 = np.concatenate([rus_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in test_loader])
test_embeds_x2 = np.concatenate([rus_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in test_loader])
test_embeds = np.concatenate([test_embeds_x1, test_embeds_x2], axis=1)
test_labels = np.concatenate([data[3].detach().cpu().numpy() for data in test_loader])
test_labels = sarcasm_label(test_labels)

clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
score

##SimCLR

In [ ]:
encoders = [Transformer(371, 40).cuda(), Transformer(300, 40).cuda()] #must have the same feature dim
supcon_model = SupConResNet(temperature=0.5, encoders=encoders, dim_ins=[40, 40], feat_dims=[40, 40]).cuda()
supcon_optim = optim.Adam(supcon_model.parameters())
train_supcon_sarcasm(supcon_model, train_loader, supcon_optim, modalities=[0,2], num_epoch=100)

In [ ]:

train_embeds_x1 = np.concatenate([supcon_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in train_loader])
train_embeds_x2 = np.concatenate([supcon_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in train_loader])
train_embeds = np.concatenate([train_embeds_x1, train_embeds_x2], axis=1)
train_labels = np.concatenate([data[3].detach().cpu().numpy() for data in train_loader])
train_labels = sarcasm_label(train_labels)
  
test_embeds_x1 = np.concatenate([supcon_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in test_loader])
test_embeds_x2 = np.concatenate([supcon_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in test_loader])
test_embeds = np.concatenate([test_embeds_x1, test_embeds_x2], axis=1)
test_labels = np.concatenate([data[3].detach().cpu().numpy() for data in test_loader])
test_labels = sarcasm_label(test_labels)

clf = LogisticRegression(max_iter=100).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)


In [ ]:
score